In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/107.7 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install telebot
import os
import telebot
import shutil

BOT_TOKEN = '7110063510:AAG_aOvVzBiFHVJIyiscKVfo9s0zoAFQ2to'
bot = telebot.TeleBot(BOT_TOKEN)
output_folder = "/content/runs/detect/predict"

@bot.message_handler(content_types=['photo'])
def echo_photo(message):
    print("Image Received")
    # Get the photo ID of the largest available photo
    photo = message.photo[-1]
    # Download the photo
    photo_info = bot.get_file(photo.file_id)
    photo_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{photo_info.file_path}"

    # Define the folder where images will be saved
    save_folder = "saved_images"
    try:
        shutil.rmtree("/content/saved_images")
        shutil.rmtree("/content/runs/detect/predict")
        shutil.rmtree("/content/runs/detect/predict2")
        shutil.rmtree("/content/runs/detect/predict3")
    except Exception as e:
        print(f"Error while removing directory: {e}")

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Define the image filename
    image_filename = f"{message.from_user.id}_{message.message_id}.jpg"
    image_path = os.path.join(save_folder, image_filename)

    os.system(f"wget -O {image_path} {photo_url}")
    !yolo task=detect mode=predict model=/content/drive/MyDrive/skin_ds_2.pt conf=0.25 source='/content/saved_images' save=True

    # Send the downloaded photo back to the user
    final_image_path = os.path.join(output_folder, image_filename)

    with open(final_image_path, 'rb') as photo_file:
        bot.send_photo(message.chat.id, photo_file)

    bot.reply_to(message, "Image saved and sent!")

@bot.message_handler(content_types=['video'])
def video_handler(message):
    print("Video Received")
    # Get the video file ID
    video = message.video
    # Download the video
    video_info = bot.get_file(video.file_id)
    video_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{video_info.file_path}"

    # Define the folder where videos will be saved
    save_folder = "saved_videos"
    try:
        shutil.rmtree("/content/saved_videos")
        shutil.rmtree("/content/runs/detect/predict")
        shutil.rmtree("/content/runs/detect/predict2")
        shutil.rmtree("/content/runs/detect/predict3")
    except Exception as e:
        print(f"Error while removing directory: {e}")

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Define the video filename
    video_filename = f"{message.from_user.id}_{message.message_id}.mp4"
    video_path = os.path.join(save_folder, video_filename)

    os.system(f"wget -O {video_path} {video_url}")
    !yolo task=detect mode=predict model=/content/drive/MyDrive/skin_ds_2.pt conf=0.25 source='/content/saved_videos' save=True

    # Send the processed video back to the user
    processed_files = os.listdir(output_folder)
    if processed_files:
        first_processed_file_path = os.path.join(output_folder, processed_files[0])
        with open(first_processed_file_path, 'rb') as video_file:
            bot.send_video(message.chat.id, video_file, supports_streaming=True)
        bot.reply_to(message, "Video processed and sent!")
    else:
        bot.reply_to(message, "No processed video files found.")

@bot.message_handler(func=lambda message: True)
def handle_text_messages(message):
    bot.reply_to(message, "Please send an image or a video.")

if __name__ == "__main__":
    bot.polling(none_stop=True)
